#  Analyse du taux de recouvrement (Tunisie) — Notebook EDA + KPI + Visualisation

> **Objectif :** préparer les données, produire des **KPI** et créer des **visualisations** (évolution temporelle, comparaison BR / régimes, heatmap).  
> **Dataset :** `dataset_recouvrement.csv` (colonnes : `_id`, `Code_BR`, `Code_Regime`, `Annees`, `T_Recouvremeant`)

---

##  0) Utilisation sur Google Colab

1. Ouvre Google Colab  
2. *File → Upload notebook* et importe ce `.ipynb`  
3. Uploade aussi le fichier `data/dataset_recouvrement.csv` sur Colab (ou mets-le dans Google Drive)
4. Exécute les cellules dans l'ordre.

> Astuce : si tu utilises Drive, remplace `DATA_PATH` par ton chemin Drive.


In [ ]:
# === 1) Imports & configuration ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

#  Chemin dataset (local/Colab)
DATA_PATH = "../data/dataset_recouvrement.csv"


## 2) Chargement des données

In [ ]:
df_raw = pd.read_csv(DATA_PATH)
print("Shape:", df_raw.shape)
display(df_raw.head())
display(df_raw.dtypes)


## 3) Data processing (nettoyage & typage)

### Ce qu'on fait ici :
- Standardiser les noms de colonnes
- Convertir `T_Recouvremeant` (ex: `81.14%`) → **numérique** (float)
- Vérifier les types (entiers pour codes/année)
- Supprimer doublons
- Créer une colonne finale : `taux_pct`


In [ ]:
df = df_raw.copy()
df.columns = [c.strip() for c in df.columns]

# Conversion % -> float
df["taux_pct"] = (
    df["T_Recouvremeant"].astype(str)
      .str.replace("%", "", regex=False)
      .str.replace(",", ".", regex=False)
)
df["taux_pct"] = pd.to_numeric(df["taux_pct"], errors="coerce")

# Typage (robuste)
for col in ["_id", "Code_BR", "Code_Regime", "Annees"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

# Doublons
before = len(df)
df = df.drop_duplicates()
print("Doublons supprimés:", before - len(df))

display(df.head())


## 4) Qualité des données (valeurs manquantes)

In [ ]:
na = (df.isna().mean() * 100).round(2).sort_values(ascending=False)
na_df = na.reset_index()
na_df.columns = ["colonne", "pourcentage_NA"]
display(na_df)

plt.figure(figsize=(10,4))
plt.bar(na_df["colonne"], na_df["pourcentage_NA"])
plt.xticks(rotation=75, ha="right")
plt.title("Valeurs manquantes par colonne")
plt.ylabel("%")
plt.tight_layout()
plt.show()


## 5) KPI Dashboard (indicateurs clés)

In [ ]:
kpi = {
    "Total lignes": len(df),
    "BR uniques": df["Code_BR"].nunique(dropna=True),
    "Régimes uniques": df["Code_Regime"].nunique(dropna=True),
    "Années uniques": df["Annees"].nunique(dropna=True),
    "Taux moyen (%)": round(df["taux_pct"].mean(), 2),
    "Taux min (%)": round(df["taux_pct"].min(), 2),
    "Taux max (%)": round(df["taux_pct"].max(), 2),
}

kpi_df = pd.DataFrame({"KPI": list(kpi.keys()), "Valeur": list(kpi.values())})
display(kpi_df)


In [ ]:
# KPI cards (visuel type dashboard)
cards = [
    ("Total", kpi["Total lignes"]),
    ("BR", kpi["BR uniques"]),
    ("Régimes", kpi["Régimes uniques"]),
    ("Moyenne %", kpi["Taux moyen (%)"]),
]
plt.figure(figsize=(12,2.4))
plt.axis("off")
x0, w = 0.02, 0.24
for i,(t,v) in enumerate(cards):
    x = x0 + i*w
    plt.gca().add_patch(plt.Rectangle((x,0.18), w-0.03, 0.68, fill=False, linewidth=1.5))
    plt.text(x+0.02, 0.75, str(t), fontsize=11, fontweight="bold")
    plt.text(x+0.02, 0.35, str(v), fontsize=16)
plt.title("Dashboard — KPI (aperçu)", pad=10)
plt.show()


## 6) Visualisations

### 6.1 Distribution du taux (%) (histogramme)

In [ ]:
plt.figure(figsize=(8,4))
plt.hist(df["taux_pct"].dropna(), bins=20)
plt.title("Distribution du taux de recouvrement (%)")
plt.xlabel("Taux (%)")
plt.ylabel("Fréquence")
plt.tight_layout()
plt.show()


### 6.2 Évolution temporelle (taux moyen par année)

In [ ]:
ts = df.groupby("Annees", dropna=True)["taux_pct"].mean().sort_index()

plt.figure(figsize=(8,4))
plt.plot(ts.index.astype(int), ts.values, marker="o")
plt.title("Évolution du taux de recouvrement moyen par année")
plt.xlabel("Année")
plt.ylabel("Taux moyen (%)")
plt.tight_layout()
plt.show()

display(ts)


### 6.3 Top 15 BR par taux moyen

In [ ]:
top_br = (
    df.groupby("Code_BR")["taux_pct"].mean()
      .sort_values(ascending=False)
      .head(15)
)

plt.figure(figsize=(10,4))
plt.bar(top_br.index.astype(str), top_br.values)
plt.xticks(rotation=75, ha="right")
plt.title("Top 15 BR — taux moyen de recouvrement")
plt.xlabel("Code_BR")
plt.ylabel("Taux moyen (%)")
plt.tight_layout()
plt.show()

display(top_br.to_frame("taux_moyen_pct"))


### 6.4 Comparaison par régime (taux moyen)

In [ ]:
reg = (
    df.groupby("Code_Regime")["taux_pct"].mean()
      .sort_values(ascending=False)
)

plt.figure(figsize=(10,4))
plt.bar(reg.index.astype(str), reg.values)
plt.xticks(rotation=75, ha="right")
plt.title("Taux moyen de recouvrement par régime")
plt.xlabel("Code_Regime")
plt.ylabel("Taux moyen (%)")
plt.tight_layout()
plt.show()

display(reg.to_frame("taux_moyen_pct"))


### 6.5 Heatmap BR × Année (Top 20 BR les plus fréquents)

In [ ]:
pivot = df.pivot_table(
    index="Code_BR",
    columns="Annees",
    values="taux_pct",
    aggfunc="mean"
)

top20_br = df["Code_BR"].value_counts().head(20).index
pivot2 = pivot.loc[pivot.index.isin(top20_br)].sort_index()

plt.figure(figsize=(10,5))
plt.imshow(pivot2.values, aspect="auto")
plt.xticks(range(len(pivot2.columns)), pivot2.columns.astype(int))
plt.yticks(range(len(pivot2.index)), pivot2.index.astype(str))
plt.colorbar(label="Taux moyen (%)")
plt.title("Heatmap — Taux moyen (%) | Top 20 BR × Années")
plt.tight_layout()
plt.show()

display(pivot2)


## 7) Exports (optionnel)

In [ ]:
# Export cleaned + KPI (utile pour Quarto/Power BI)
df.to_csv("dataset_recouvrement_clean.csv", index=False, encoding="utf-8")
kpi_df.to_csv("kpi_recouvrement.csv", index=False, encoding="utf-8")
print(" Exports: dataset_recouvrement_clean.csv, kpi_recouvrement.csv")
